In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

In [2]:
key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
load_dotenv()

True

In [4]:
from langchain_community.llms import HuggingFaceHub

In [5]:
key = os.getenv('HUGGINGFACEHUB_API_KEY')

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [7]:
print(llm)

model='gemini-pro' client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)


In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text: {text}
You are an expert MCQ Maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

{response_json}
"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [12]:
quiz_chain = LLMChain(llm = llm,
                      prompt = quiz_generation_prompt,
                      output_key='quiz',
                      verbose = True)

In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template=TEMPLATE2
)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, 
                      output_key="review", verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], 
                                        input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path = r'C:\Users\Manan Bedi\Documents\LLM_iNeuron\MCQ_Generator\data.txt'

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [20]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [22]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )


c:\anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin

In [23]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [24]:
quiz=response.get("quiz")

In [25]:
quiz[15:]

' {\n    "1": {\n        "mcq": "What characterizes biology as a scientific study?",\n        "options": {\n            "a": "Focus on living organisms",\n            "b": "Use of the scientific method",\n            "c": "Both a and b",\n            "d": "Neither a nor b"\n        },\n        "correct": "c"\n    },\n    "2": {\n        "mcq": "Which of the following is a unifying theme in biology?",\n        "options": {\n            "a": "Reproduction",\n            "b": "Evolution",\n            "c": "Heredity",\n            "d": "All of the above"\n        },\n        "correct": "d"\n    },\n    "3": {\n        "mcq": "What is the primary function of energy processing in organisms?",\n        "options": {\n            "a": "Growth",\n            "b": "Reproduction",\n            "c": "Movement",\n            "d": "All of the above"\n        },\n        "correct": "d"\n    },\n    "4": {\n        "mcq": "Which of the following is NOT a subdiscipline of biology?",\n        "options":

In [26]:
quiz=json.loads(quiz[15:])

In [27]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [28]:
quiz_table_data

[{'MCQ': 'What characterizes biology as a scientific study?',
  'Choices': 'a: Focus on living organisms | b: Use of the scientific method | c: Both a and b | d: Neither a nor b',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is a unifying theme in biology?',
  'Choices': 'a: Reproduction | b: Evolution | c: Heredity | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'What is the primary function of energy processing in organisms?',
  'Choices': 'a: Growth | b: Reproduction | c: Movement | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'Which of the following is NOT a subdiscipline of biology?',
  'Choices': 'a: Molecular biology | b: Geology | c: Anatomy | d: Population evolution',
  'Correct': 'b'},
 {'MCQ': 'What is the first step in the scientific method?',
  'Choices': 'a: Forming a hypothesis | b: Making observations | c: Performing experiments | d: Drawing conclusions',
  'Correct': 'b'}]

In [29]:
quiz=pd.DataFrame(quiz_table_data)

In [30]:
quiz.to_csv("mcq.csv",index=False)

In [31]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_07_2024_19_26_02'